# Analyse ADN MyHeritage

---

**Site d'analyse :**

- MyHeritage

- MyTrueAncestry

rsid = ID SNP de référence (indique le « nom » du premier et du dernier SNP du segment)

Les SNP (pour single nucleotide polymorphism), prononcés Snips, correspondent à des variations mineures du génome au sein d'une population. Un seul nucléotide, le composant de base de l'ADN, est modifié.

Voir les Étude d'association pangénomique

https://towardsdatascience.com/know-thyself-using-data-science-to-explore-your-own-genome-ec726303f16c

https://towardsdatascience.com/machine-learning-in-bioinformatics-genome-geography-d1b1dbbfb4c2

In [2]:
# data visualization
import seaborn as sns
sns.set_style('darkgrid')
sns.color_palette('Spectral')
import matplotlib.pyplot as plt

# data analysis
import numpy as np
import requests
import pandas as pd
import re

# web scrapping
import sys
import requests
from bs4 import BeautifulSoup as bs

## Data collection

In [3]:
data = pd.read_csv('MyHeritage_raw_dna_data.csv', sep=',', dtype={'RSID':'str', 'CHROMOSOME':'object', 'POSITION':'int', 'RESULT':'str'}, comment='#', low_memory=False)
df = pd.DataFrame(data) # for Collection and EDA
data.head()

,RSID,CHROMOSOME,POSITION,RESULT
0,rs547237130,1,72526,AA
1,rs562180473,1,565703,AA
2,rs575203260,1,567693,TT
3,rs3131972,1,752721,AA
4,rs200599638,1,752918,GG


**Description des variables :**

(RESULT is Genotype)

In [4]:
display(df.isna().any(), df.nunique())

RSID          False
CHROMOSOME    False
POSITION      False
RESULT        False
dtype: bool

RSID          609346
CHROMOSOME        24
POSITION      608314
RESULT            14
dtype: int64

**Simplification du dataset :**

In [5]:
# replace X and Y by values (regex using)
chromosome_dict = df['CHROMOSOME'].unique()
chromosome_dict = dict(zip(list(range(chromosome_dict.size)), chromosome_dict))
df['CHROMOSOME'] = df['CHROMOSOME'].apply(lambda x: re.sub(r'X', r'23', x))
df['CHROMOSOME'] = df['CHROMOSOME'].apply(lambda x: re.sub(r'Y', r'24', x))
df['CHROMOSOME'] = df['CHROMOSOME'].apply(lambda x:int(x))
df.head()

,RSID,CHROMOSOME,POSITION,RESULT
0,rs547237130,1,72526,AA
1,rs562180473,1,565703,AA
2,rs575203260,1,567693,TT
3,rs3131972,1,752721,AA
4,rs200599638,1,752918,GG


**Acquisition de la base de donnée SNPedia (https://www.snpedia.com/)** 

Article interessant : https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3245045/

tuto wikitool (https://www.snpedia.com/index.php/Bulk), ne marche pas avec Python3, faire webscrapping avec BeautifulSoup. Voir :

- https://stackoverflow.com/questions/70233801/python-scraping-of-wikipedia-category-page
- https://stackoverflow.com/questions/62398524/how-can-i-get-an-article-from-wiki-with-a-specific-language-using-python

***Remarque sur la base de donnée :***

La base ClinVar est pour l'etude des variations de phenotype (dans notres cas, uniquement les maladies, GWAS plus synthétique et le tableau est plus simple à traiter). De maniere générale :

**Clinvar** : La majorité des études rapportées sont de la forme "nous examinons 50 personnes atteintes d'une maladie rare X et avons trouvé une forte prévalence de la variation rare Y, donc la variation Y est très probablement pathogène". Les rapports de cotes sont rarement rapportés, j'imagine parce que la variation est si rare qu'il serait impossible de trouver suffisamment de contrôles pour calculer cela avec une quelconque signification statistique.

**GWAS** . La majorité des tests sont de la forme "nous avons examiné les génomes de 2000 personnes pour toute variation génomique qui pourrait expliquer la prévalence de la condition Y. Nous avons constaté que les personnes avec la variation X avaient 1,3 fois plus de chances de développer la condition Y que celles sans variation X.". La plupart de ces résultats font état de variations relativement courantes, probablement parce que cela conduit à des résultats statistiquement significatifs compte tenu de la taille de l'échantillon.

In [ ]:
nb_snp = 111727
snp_list = []
# get query request loop
URL = "http://bots.snpedia.com/api.php"
for i in range(int(nb_snp/500)+1) :
    if i == 0 :
        PARAMS = { "action": "query","cmtitle": "Category:Is a snp", "cmlimit": '500' , "list": "categorymembers","format": "json"}
    else :
        PARAMS = { "action": "query","cmtitle": "Category:Is a snp", 'cmcontinue': lpage, "cmlimit": '500' , "list": "categorymembers","format": "json"}
    # request
    req = requests.get(url=URL, params=PARAMS).json()
    if 'continue' in req.keys(): lpage = req['continue']['cmcontinue']
    # get list
    pages = req['query']['categorymembers']
    snp_list += [p['title'] for p in pages]
#display(snp_list)

In [5]:
gwas_list = []
# get query request loop
URL = "http://bots.snpedia.com/api.php"
PARAMS = { "action": "query","cmtitle": "Category:GWAS", "cmlimit": '500' , "list": "categorymembers","format": "json"}
# request
req = requests.get(url=URL, params=PARAMS).json()
# get list
pages = req['query']['categorymembers']
gwas_list += [p['title'] for p in pages]
display(gwas_list)

['MYH6 c.2161',
 'Rs1000113',
 'Rs10033464',
 'Rs10045431',
 'Rs1004819',
 'Rs10078095',
 'Rs1012053',
 'Rs1015362',
 'Rs10260404',
 'Rs1038304',
 'Rs1042602',
 'Rs1042725',
 'Rs10483853',
 'Rs10486567',
 'Rs10486776',
 'Rs10488360',
 'Rs10492604',
 'Rs10493340',
 'Rs10493485',
 'Rs10494366',
 'Rs10496265',
 'Rs10497721',
 'Rs10498345',
 'Rs10498792',
 'Rs10499194',
 'Rs10499559',
 'Rs10501570',
 'Rs10501920',
 'Rs10503887',
 'Rs10505477',
 'Rs10506821',
 'Rs10507380',
 'Rs10510634',
 'Rs10514345',
 'Rs10516487',
 'Rs10516541',
 'Rs1051730',
 'Rs10757278',
 'Rs10778213',
 'Rs10789230',
 'Rs10795668',
 'Rs10798269',
 'Rs10801047',
 'Rs10811661',
 'Rs10871290',
 'Rs10883365',
 'Rs10889676',
 'Rs10889677',
 'Rs10896449',
 'Rs10911902',
 'Rs10945919',
 'Rs10946398',
 'Rs10946808',
 'Rs10984447',
 'Rs10993994',
 'Rs10994336',
 'Rs11052552',
 'Rs11064768',
 'Rs1106683',
 'Rs1106684',
 'Rs11101442',
 'Rs1111875',
 'Rs11171739',
 'Rs11200638',
 'Rs11209002',
 'Rs11209003',
 'Rs11209026',
 'Rs1

In [ ]:
# flip - flop choice
rsid_list = gwas_list # or snp_list

In [8]:
filename = "gwas" # or referenced if snp

**Correspondance entre la liste et le dataframe (réduction) :**

In [7]:
snp_series = pd.Series(rsid_list,  name="RSID")
snp_series = snp_series.apply(lambda x: re.sub(r'R', r'r', x))
# new df
snp_df = pd.concat([snp_series,pd.Series(range(len(snp_series)), name="ID_LIST")], axis=1)
snp_df = snp_df.merge(df, how='inner', on=['RSID'])
snp_df.head()
# save list of rsid referenced
snp_df[['ID_LIST','RSID']].to_csv('rsid_'+ filename +'.csv', index=False)

In [9]:
# checkpoint
snp_df = pd.read_csv('rsid_'+ filename +'.csv')

**Parsing des pages rsid (GWAS table) :**

In [25]:
def extract_table(URL, page):
    PARAMS = {"action": "parse", "page": page,"prop": "text", "section": 0, "format": "json"}
    # prettify request
    results = requests.get(url=URL, params=PARAMS).json()
    soup = bs(results['parse']['text']['*'], 'html.parser')
    # find body of each table
    tables = soup.find_all("tbody")
    for t in tables :
        head = t.select("th")
        if head :
            h_text = head[0].getText("", strip=True)
            if h_text == "GWAS" :
                # extract info
                body = t.select("td")
                # dict
                data = {}
                for h,b in zip(head[1:], body):
                    data[h.getText("", strip=True)] = b.getText("", strip=True)
    return data

In [27]:
URL = "http://bots.snpedia.com/api.php"
data = []
for p in snp_df.RSID :
    try :
        d = extract_table(URL,p)
        data += [d]
        print(p)
    except :
        print(p + " -- Unexpected error:", sys.exc_info()[0])
# save
gwas_df = pd.DataFrame(data)
gwas_df.to_csv('GWAS_DATA.csv', index=False)

rs1000113
rs10033464
rs10045431
rs1004819
rs10078095
rs1012053
rs1015362
rs10260404
rs1038304
rs1042602
rs1042725
rs10488360
rs10492604
rs10493340
rs10494366
rs10496265
rs10497721
rs10498792
rs10499194
rs10499559
rs10503887
rs10505477
rs10510634
rs10514345
rs10516487
rs10516541
rs1051730
rs10757278
rs10778213
rs10789230
rs10795668
rs10801047
rs10811661
rs10883365
rs10889676
rs10889677
rs10896449
rs10945919
rs10946398
rs10946808
rs10984447
rs10993994
rs10994336
rs11101442
rs1111875
rs11171739
rs11200638
rs11209002
rs11209003
rs11209026
rs11465802
rs11465804
rs1155865
rs11574637
rs1158167
rs1169310
rs11761231
rs12130333
rs1219648
rs12198986
rs12203592
rs12290811
rs12304921
rs12567232
rs12593813
rs12680546
rs12708716
rs12722489
rs12779790
rs12821256
rs12899449
rs12913832
rs12949531
rs12970134
rs13015714
rs13266634
rs13277113
rs13281615
rs1333026
rs1333049
rs13361189
rs13387042
rs13397985
rs1343151
rs1344706
rs1373692
rs1398024
rs1408799
rs1412337
rs1427407
rs1492820
rs1495377
rs1540771
rs

In [10]:
# checkpoint
gwas_df = pd.read_csv('GWAS_DATA.csv')
gwas_df.head()

,SNP,PubMedID,Condition,Gene,Risk Allele,pValue,OR,95% CI
0,rs1000113,[PMID 17554300],Crohn's disease,IRGM,T,3.000000e-07,1.54,1.31-1.82
1,rs10033464,[PMID 17603472],Atrial fibrillation/atrial flutter,"PITX2,ENPEP",T,7.000000e-11,1.39,1.26-1.53
2,rs10045431,[PMID 18587394],Crohn's disease,IL12B,C,4.000000e-13,1.11,NaN
3,rs1004819,[PMID 17804789],Crohn's disease,IL23R,NaN,1.000000e-08,1.38,1.23-1.53
4,rs10078095,[PMID 18193045],Height,HOMER1,C,3.000000e-06,0.90,NR) cm talle


## Data Analysis


In [15]:
df_exp = gwas_df.copy()
df_exp.rename(columns={'SNP': 'RSID'}, inplace=True)

**Déterminer les données à analyser :**

In [16]:
# risk is relevant
isna = df_exp['Risk Allele'].isna().sum()
notna = df_exp['Risk Allele'].notna().sum()
display(isna,notna)

92

224

In [17]:
df_exp = df_exp[df_exp['Risk Allele'].notna()]
# condition diversity number
Condition = df_exp.groupby(['Condition']).apply(lambda x:x.index).index
print(Condition.size)

68


**Combiner les données MyHeritage avec Database :**

In [18]:
df_exp = df_exp.merge(df, how='inner', on=['RSID'])
df_exp.head()

,RSID,PubMedID,Condition,Gene,Risk Allele,pValue,OR,95% CI,CHROMOSOME,POSITION,RESULT
0,rs1000113,[PMID 17554300],Crohn's disease,IRGM,T,3.000000e-07,1.54,1.31-1.82,5,150240076,CT
1,rs10033464,[PMID 17603472],Atrial fibrillation/atrial flutter,"PITX2,ENPEP",T,7.000000e-11,1.39,1.26-1.53,4,111720761,GT
2,rs10045431,[PMID 18587394],Crohn's disease,IL12B,C,4.000000e-13,1.11,NaN,5,158814533,CC
3,rs10078095,[PMID 18193045],Height,HOMER1,C,3.000000e-06,0.90,NR) cm talle,5,78756778,CC
4,rs1012053,[PMID 17486107],Bipolar disorder,DGKH,A,2.000000e-08,1.59,1.35-1.87,13,42653437,AA


**Calculer le nombre d'Allele de risque :**

Mieux comprendre https://www.ebi.ac.uk/gwas/ (+ completer database si besoin)

In [31]:
def custom_row(row):
    return row[0].count(row[1])

df_exp['Count'] = df_exp[['RESULT','Risk Allele']].apply(custom_row, axis=1)
#df_exp[['RESULT','Risk Allele']].apply(lambda x : x[0].count(x[1]), axis=1)

**Calculer la proportion totale par condition :**

In [48]:
group_condition = df_exp.groupby(['Condition'])
# simplify data
listing = [['Condition','Proportion']]
for c,g in group_condition :
    allele_total = g['Count'].sum()
    prop = allele_total / (2*g.shape[0])
    # increment data
    listing += [[c,prop]]

**Enregistrement des resultats provisoire (synthèse) :**

In [61]:
df_analysis = pd.DataFrame(listing[1:], columns=listing[0])
# minimize propability (to define)
df_analysis.Proportion = df_analysis.Proportion/1.66
# Visualization
df_analysis.Proportion = df_analysis.Proportion.map(lambda n: '{:,.2%}'.format(n))
df_analysis.head()

,Condition,Proportion
0,APOE*e4 carriers with late onset Alzheimer's d...,0.00%
1,Age-related macular degeneration,30.12%
2,Asthma,30.12%
3,Atrial fibrillation/atrial flutter,15.06%
4,Bipolar disorder,25.10%


In [62]:
df_analysis.to_csv('GenomeAnalysis_provisional.csv', index=False)